In [1]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

mnist = load_digits()
x, tx, y_label, ty_label = train_test_split(mnist.images, mnist.target, test_size=0.2)
x, tx = x/255.0, tx/255.0
x = x.reshape([x.shape[0], 64])
tx = tx.reshape([tx.shape[0], 64])

def getOneHot(y_label):
    y = []
    for yl, i in enumerate(y_label):
        onehot_vector = np.zeros([10])
        onehot_vector[i] = 1
        y.append(onehot_vector)
    y = np.array(y)
    return y

y = getOneHot(y_label)
ty = getOneHot(ty_label)


In [ ]:
from NN import RBFNetwork
from NN.loss import MeanSquaredError
from NN.optimizer import Adam

rbf = RBFNetwork(64, 7, 10, 30, MeanSquaredError(), Adam(1e-3))

rbf.train(x, y, 1000, 1)
print('test accuracy', rbf.get_accuracy(tx, ty))


In [ ]:
# # # test for tensorflow
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, Dense, Flatten, ReLU, Reshape
# from tensorflow.keras.losses import CategoricalCrossentropy
# import tensorflow as tf

# model = Sequential(
#     [
#         Flatten(),
#         Dense(64, activation='relu'),
#         Reshape([8, 8, 1]),
#         Conv2D(3, 3, 1, 'same', activation='relu'),
#         Conv2D(6, 3, 2, 'same', activation='relu'),
#         Flatten(),
#         Dense(10, activation='relu'),
#         Dense(10, activation='softmax'),
#     ]
# )

# model.compile(optimizer='adam',
#              loss=CategoricalCrossentropy(),
#              metrics=['accuracy'])

# with tf.device('/cpu:0'):
#     model.fit(x, y, 32, epochs=200)

# target = model.predict(tx)
# ptx = target.argmax(axis=1)
# print('accuracy:', accuracy_score(ptx, ty_label))
# confusion_matrix(ptx, ty_label)


In [2]:
from NN.network import Network
from NN.layer import DenseLayer, Flatten, Conv2D, Reshape
from NN.layer.activation import Sigmoid, Softmax, Tanh, ReLU
from NN.loss import CategoricalCrossEntropy
from NN.optimizer import SGD


# hyperparameters
n_filters = 6
kernel_len = 3
stride_len = 2
kernel_size = [kernel_len, kernel_len]
stride_size = [stride_len, stride_len]
dilation_size = [1, 1]
padding = 'same'
image_shape= [8, 8]

net = Network([
    Flatten(image_shape),
    DenseLayer(64, 64),
    Tanh(),
    Reshape([64], [8, 8, 1]),
    Conv2D(input_shape=[8, 8, 1],
            n_filters=n_filters,
            kernel_size=kernel_size,
           strides=stride_size,
            dilation_rate=dilation_size,
            padding=padding),
    Tanh(),
    Conv2D(input_shape=[4, 4, n_filters],
            n_filters=n_filters*2,
            kernel_size=kernel_size,
            strides=stride_size,
            dilation_rate=dilation_size,
            padding=padding),
    Tanh(),

    Flatten([2, 2, 2*n_filters]),
    DenseLayer(4*2*n_filters, 10),
    ReLU(),
    
    DenseLayer(10, 10),
    Softmax(),
], optimizer=SGD(1e-3),
   loss_func=CategoricalCrossEntropy(),
)
net.Train(x, y, batchsize=32, epochs=200, verbose=3)


ModuleNotFoundError: jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.

In [ ]:
target = net.Predict(tx)
ptx = target.argmax(axis=1)
print('accuracy:', accuracy_score(ptx, ty_label))
confusion_matrix(ptx, ty_label)


In [ ]:
def show_test_img(index):
    plt.xlabel("predicted: " + str(ptx[index]))
    plt.title('answer: ' + str(ty_label[index]))
    plt.imshow(tx[index].reshape([8, 8]), cmap='gray')


def show(index):
    print(ty_label[index], ptx[index])
    print(target[index])
    print(ty[index])
